In [5]:
import os
import glob
import cv2
import numpy as np
import torch
from unet_model import UNet

# ---------- Configuration ----------
image_dir = "../../data/test/inputs"
mask_dir = "../../data/test/masks/unet"
model_path = "best_unet.pth"
input_size = (768, 512)  # (width, height)

os.makedirs(mask_dir, exist_ok=True)

# ---------- Load Model ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ---------- Prediction ----------
print(device)


cuda


In [7]:
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.tiff")))

for img_path in image_paths:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, input_size).astype(np.float32) / 255.0
    tensor = torch.tensor(img_resized).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(tensor)
        pred_mask = (pred.squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255

    # Resize back to original size (optional)
    pred_mask_resized = cv2.resize(pred_mask, (img.shape[1], img.shape[0]))

    # Save output
    filename = os.path.basename(img_path)
    out_path = os.path.join(mask_dir, filename)
    cv2.imwrite(out_path, pred_mask_resized)

print("Inference completed. Masks saved to", mask_dir)

Inference completed. Masks saved to ../../data/test/masks/unet
